## Use metadata to filter out full datasets

From Felix:

The id is the value that later ends up in observation, and denotes a unique dataset. The payload designates a given data transmission as either incremental, meaning that it’s a slice of data that’s transmitted during the experiment, or full, meaning that it contains the entire dataset to that point (we transmit the full dataset upon completion of the experiment). For incremental datasets, the slice indicates the first row from the dataset contained in that transmission.


So the idea here is to convert the `metadata` which is a series of STRING dictonaries <sup>[1](#myfootnote1)</sup> (those are dictionaries but saved as strings) into a DF. Following that to get the corresponding id or row slice number and select this row from the data list of JSONs.


 
 <a name="myfootnote1">1</a>: For example see the following output:

```
surveys_df['metadata'][0]

'{"slice":47,"id":"17734ede-5c5d-4c50-a48e-2acfaac1aa68","payload":"incremental"}'
```

# Follow Felix's R code

## Import data from SQLite file

In [1]:
import pandas as pd
import sqlite3
import numpy as np

con = sqlite3.connect("./data.sqlite")

# Load all the data into a DataFrame
database_d = pd.read_sql_query("SELECT * from labjs", con)

# Be sure to close the connection
con.close()

## Convert JSON data to table

### Extract metadata

In [2]:
# create a dataframe from the metadata column
database_d_meta = database_d['metadata'].apply(eval).apply(pd.Series).rename(columns={"id":"observation"})

In [3]:
database_d_meta

,slice,observation,payload
0,0,0a460f84-7833-4d43-b505-96d04bdf76d5,full
1,0,0a460f84-7833-4d43-b505-96d04bdf76d5,incremental


### Merge metadata back

In [4]:
# merge the two and remove metadata column
database_d = database_d.join(database_d_meta).drop("metadata", axis=1)

In [5]:
database_d

,id,session,timestamp,url,data,slice,observation,payload
0,1,o5pbac2mcj749kscpssso4f56d,2021-06-08T09:32:43+00:00,http://face-experiment.weizmann.ac.il/3/,"[{""url"":[],""meta"":{""labjs_version"":""20.2.2"",""l...",0,0a460f84-7833-4d43-b505-96d04bdf76d5,full
1,2,o5pbac2mcj749kscpssso4f56d,2021-06-08T09:32:43+00:00,http://face-experiment.weizmann.ac.il/3/,"[{""url"":[],""meta"":{""labjs_version"":""20.2.2"",""l...",0,0a460f84-7833-4d43-b505-96d04bdf76d5,incremental


### Shorten the random ids

To check how many letters in the ID we need to have as short as possible but unique identifiers, it goes over the ID and observation column and checks if number of unique parts of the string equals to number of the unique whole tags.

We are looking for a shortest sequence -- what is the minimal number of characters which will preserve how many unique inputs there is.

In [6]:
for i in range(5,37):
    if (database_d.session.str.slice(stop=i).unique().size == database_d.session.unique().size ) \
    and (database_d.observation.str.slice(stop=i).unique().size==database_d.observation.unique().size):
        print(i)        
        # if found the minimal code, replace the original session and observation with this shorted code
        database_d.session = database_d.session.str.slice(stop=i)
        database_d.observation = database_d.observation.str.slice(stop=i)
        break


5


In [7]:
database_d

,id,session,timestamp,url,data,slice,observation,payload
0,1,o5pba,2021-06-08T09:32:43+00:00,http://face-experiment.weizmann.ac.il/3/,"[{""url"":[],""meta"":{""labjs_version"":""20.2.2"",""l...",0,0a460,full
1,2,o5pba,2021-06-08T09:32:43+00:00,http://face-experiment.weizmann.ac.il/3/,"[{""url"":[],""meta"":{""labjs_version"":""20.2.2"",""l...",0,0a460,incremental


### Prepare to extract JSON data

### Extract complete datasets

In [8]:
completed_idx = database_d.loc[database_d.payload == 'full'].index
print(completed_idx)
completed_idx.size

Int64Index([0], dtype='int64')


1

In [9]:
# if no experiment ended on full, skip this part
if not completed_idx.empty:
    # get all the metadata which end up on full AND add the observation column to each register
    database_d_full = pd.concat([pd.read_json(database_d['data'].iloc[i]).join(
        pd.DataFrame({'observation': [database_d.loc[i].observation]})) for i in completed_idx])

    # unpack the first JSON meta layer
    temp = pd.DataFrame.from_dict(database_d_full.meta.apply(pd.Series)).drop(0, axis=1)

    # Unpack the inner meta layer, add prefix
    temp = pd.concat([temp, temp['labjs_build'].apply(pd.Series).drop(0, axis=1)],
                     axis=1).drop('labjs_build', axis=1).add_prefix("labjs_build_")

    # concat with the original df and drop id and data columns and rename the empty col to be called x as in the R version
    # also add a prefix meta_ to all the meta columns
    database_d_full = pd.concat([database_d_full, temp.add_prefix("meta_")], axis=1).rename(columns={"": "x"}).drop(
        "meta", axis=1)

    # add observation to all columns
    database_d_full.observation.fillna(method="ffill", inplace=True)
    
else:
    database_d_full=pd.DataFrame({"observation":[]})
    print("No experiment ended on full!")

### Extract incremental datasets

do the exact same thing but with the refisters which did not end up on full and are only partial

In [10]:
temp = None
# get all the metadata which does not end up on full AND add the observation column to each register
database_d_incremental = pd.concat([pd.read_json(database_d['data'].iloc[i]).join(
    pd.DataFrame({'observation': database_d.loc[i].observation}, index=[0])) for i in \
                                    database_d.loc[database_d.payload.isin(['latest', 'incremental'])].index])

# unpack the first meta layer
temp = pd.DataFrame.from_dict(database_d_incremental.meta.apply(pd.Series)).drop(0, axis=1)

# Unpack the inner layer, add prefix
temp = pd.concat([temp, temp['labjs_build'].apply(pd.Series).drop(0, axis=1)],
                 axis=1).drop('labjs_build', axis=1).add_prefix("labjs_build_")

# concat with the original df and drop id and data columns and rename the empty col to be called x as in the R version
# also add a prefix meta_ to all the meta columns
database_d_incremental = pd.concat([database_d_incremental, temp.add_prefix("meta_")], axis=1).rename(
    columns={"": "x"}).drop("meta", axis=1)


# add observations to all rows
database_d_incremental.observation.fillna(method="ffill", inplace=True)

### Merge datasets

In [11]:
database_d_output = pd.concat([database_d_full, \
               database_d_incremental[~database_d_incremental.observation.isin(database_d_full.observation)]])

In [12]:
# If there is a userID column defined, show how many and which users we have
if "userID" in database_d_output.columns:
    print(f"There are {database_d_output.userID.unique().shape} participants with the following names:")
    print(database_d_output.userID.unique())

## Postprocessing

### Fill nans

In [13]:
if "code" in database_d_full.columns:
    database_d_full['code'].fillna(method='ffill')

### Remove sensitive data

In [14]:
if "Email" in database_d_full.columns:
    database_d_full.columns.drop("Email", axis=1)

## Save dataset as csv

In [15]:
name = 'Felix'
database_d_full.to_csv(name+'_database_full.csv')
database_d_incremental.to_csv(name+'_database_incremental.csv')

#### End of Felix's